# Import packages

First, we import the necessary packages for this exercise.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import cartopy.feature as cfeature

# Load in the dataset(s)

In [ ]:
def preprocess_dataset(ds, tstart, tstop):
    ds = ds.sel(time=slice(tstart, tstop))
    tas_vars = ["tas", "tasmax", "tasmin"]
    for tas_var in tas_vars:
        if tas_var in ds:
            ds[tas_var].values = ds[tas_var].values - 273.15
            ds[tas_var].attrs["units"] = "°C"
    if "pr" in ds:
        ds.pr.values = ds.pr.values * 3600
        ds.pr.attrs["units"] = "mm/h"

    return ds

In [ ]:
run_name = "baseline"
run_info_list = [
    ("2021070100", 384),
    ("2021071000", 168),
    ("2021071012", 156),
    ("2021071100", 144),
    ("2021071112", 144),
    ("2021071200", 144),
    ("2021071212", 144),
]
tstart = "2021-07-13T09"
tstop = "2021-07-15T08"

In [ ]:
ds_list = list()

for run_info in run_info_list:
    rstart, nhours = run_info
    filepath = f"../data/{run_name}_{rstart}_{nhours}*.nc"
    ds = xr.open_mfdataset(filepath, engine="netcdf4", chunks="auto").assign_coords(
        rstart=rstart
    )
    ds = preprocess_dataset(ds, tstart, tstop)
    ds_list.append(ds)

ds = xr.concat(ds_list, dim="rstart")
ds.load()

# Plot the 48-hour accumulated precipitation

In [ ]:
pr = ds.pr
pr_sum = pr.sum("time", skipna=False)

ncols = len(run_info_list)
fig, axs = plt.subplots(
    ncols=ncols,
    figsize=(ncols * 3.2, 4.8),
    layout="constrained",
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for idx in range(len(run_info_list)):
    rstart, _ = run_info_list[idx]
    ax = axs[idx]
    p = pr_sum.sel(rstart=rstart).plot(
        ax=ax,
        x="lon",
        y="lat",
        levels=np.arange(0, 251, 25),
        add_colorbar=False,
        extend="max",
    )
    ax.set_title(
        f"Run start = {rstart[:4]}-{rstart[4:6]}-{rstart[6:8]}T{rstart[8:]}\nMaximum value = {pr_sum.sel(rstart=rstart).max().values:.0f} mm"
    )
    ax.coastlines(alpha=0.7)
    ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")
    ax.set_aspect(1.4)

fig.colorbar(p, label="Precipitation depth [mm]")
fig.suptitle(
    f"Accumulated precipitation between {tstart} and {tstop}", fontsize="xx-large"
)
plt.show()

# Mask with RMI warning levels

In [ ]:
def mask_pr(pr, thresholds_list):
    mask_list = list()

    for idx in range(len(thresholds_list)):
        hours, thresh = thresholds_list[idx]

        pr_rolling_sum = pr.rolling(time=hours).sum()
        mask = (pr_rolling_sum > thresh).any("time")

        mask_list.append(mask.assign_coords(idx=idx))

    mask = xr.concat(mask_list, dim="idx").any("idx")
    return pr.where(mask)

In [ ]:
yellow_thresholds_list = [(1, 20), (6, 20), (24, 25)]
orange_thresholds_list = [(1, 30), (6, 40), (24, 50)]
red_thresholds_list = [(1, 50), (6, 60), (24, 100)]

pr = ds.pr
pr_masked = mask_pr(pr, red_thresholds_list)

In [ ]:
pr_masked_sum = pr_masked.sum("time", skipna=False)

ncols = len(run_info_list)
fig, axs = plt.subplots(
    ncols=ncols,
    figsize=(ncols * 3.2, 4.8),
    layout="constrained",
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for idx in range(len(run_info_list)):
    rstart, _ = run_info_list[idx]
    ax = axs[idx]
    p = pr_masked_sum.sel(rstart=rstart).plot(
        ax=ax,
        x="lon",
        y="lat",
        levels=np.arange(0, 251, 25),
        add_colorbar=False,
        extend="max",
    )
    ax.set_title(
        f"Run start = {rstart[:4]}-{rstart[4:6]}-{rstart[6:8]}T{rstart[8:]}\nMaximum value = {pr_masked_sum.sel(rstart=rstart).max().values:.0f} mm"
    )
    ax.coastlines(alpha=0.7)
    ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")
    ax.set_aspect(1.4)

fig.colorbar(p, label="Precipitation depth [mm]")
fig.suptitle(
    f"Accumulated precipitation between {tstart} and {tstop} (red warning)",
    fontsize="xx-large",
)

# Open CLIMATE_GRID data

In [ ]:
filepath = "../data/CLIMATE_GRID_pr_1951_2023_daily_latlon_5km.nc"
obs = xr.open_dataset(filepath, engine="netcdf4", chunks="auto")
obs = preprocess_dataset(obs, tstart="2021-07-13", tstop="2021-07-14") * 24  # !!!

obs_pr = obs.pr
obs_pr_sum = obs_pr.sum("time", skipna=False)
obs_pr_sum.plot(levels=np.arange(0, 251, 25))

In [ ]:
ncols = len(run_info_list)
fig, axs = plt.subplots(
    ncols=ncols,
    figsize=(ncols * 3.2, 4.8),
    layout="constrained",
    subplot_kw={"projection": ccrs.PlateCarree()},
)

pr_sum = ds.pr.sum("time", skipna=False)
obs_pr_sum = obs.pr.sum("time", skipna=False)
bias_pr_sum = pr_sum - obs_pr_sum

for idx in range(len(run_info_list)):
    rstart, _ = run_info_list[idx]
    ax = axs[idx]
    p = bias_pr_sum.sel(rstart=rstart).plot(
        ax=ax,
        x="lon",
        y="lat",
        levels=[-200, -100, -50, -20, -10, 10, 20, 50, 100, 200],
        cmap="coolwarm_r",
        add_colorbar=False,
        extend="both",
    )
    ax.set_title(
        f"Run start = {rstart[:4]}-{rstart[4:6]}-{rstart[6:8]}T{rstart[8:]}\nMaximum bias = {bias_pr_sum.sel(rstart=rstart).max().values:.0f} mm\nMinimum bias = {bias_pr_sum.sel(rstart=rstart).min().values:.0f} mm\nMean bias = {bias_pr_sum.sel(rstart=rstart).mean().values:.0f} mm"
    )
    ax.coastlines(alpha=0.7)
    ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")
    ax.set_aspect(1.4)

fig.colorbar(p, label="Precipitation depth [mm]")
fig.suptitle(
    f"Accumulated precipitation between {tstart} and {tstop}", fontsize="xx-large"
)

Taking the average bias has an inherent issue? What is this and how can this be helped?

# Select data in a point

In [ ]:
lat, lon = 50.7941, 4.3572  # RMI
lat, lon = 50.57506002330523, 5.800553217487655  # Pepinster
pr_point = ds.pr.sel(lat=lat, lon=lon, method="nearest")
pr_point.cumsum(dim="time").plot(x="time", hue="rstart")
obs_pr_point = obs.pr.sel(lat=lat, lon=lon, method="nearest")
obs_pr_point.cumsum(dim="time").plot(x="time", hue="rstart")